In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import torch
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Tile_Dataset import Vesuvius_Tile_Datamodule
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
import einops
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")


2023-05-04 17:44:29,912 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpcbcfqly_
2023-05-04 17:44:29,913 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpcbcfqly_/_remote_module_non_scriptable.py


In [3]:
PATCH_SIZE = 224
Z_DIM = 8

class CFG:
    
    train_fragment_id=[2,3]
    val_fragment_id=[1]
    batch_size = 32
    patch_size = PATCH_SIZE
    z_dim = Z_DIM
    stride = patch_size // 2
    #comp_dataset_path = COMPETITION_DATA_DIR
    num_workers = 0
    on_gpu = True

In [4]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

dataloader = iter(dataset.train_dataloader())
for i in range(1):
    # Get image and label from train data -- change number for different ones
    subvolumes, inklabels = next(dataloader)
    print('subvolume shape:',subvolumes.shape)
    print('inklabel shape:',inklabels.shape)
    for subvolume,  inklabel in zip(subvolumes, inklabels):
                fig, axes = plt.subplots(1, 2, figsize=(15, 5))
                for idx, image in enumerate((subvolume,  inklabel)):
                    if idx==0:
                        axes[idx].imshow(image[0])
                    else:
                         axes[idx].imshow(image.squeeze(0))

                plt.show()
    

plot_dataset = dataset.train_dataloader()
plot_count = 0
for i in range(1000):
    image, mask = plot_dataset[i]
    #data = transform(image=image, mask=mask)
    aug_image = image#.squeeze(0) # data['image']
    aug_mask = mask#.squeeze(0) #data['mask']
    print(image.shape, mask.shape)

    if mask.sum() == 0:
        continue

    fig, axes = plt.subplots(1, 4, figsize=(15, 8))
    axes[0].imshow(image[..., 0], cmap="gray")
    axes[1].imshow(mask, cmap="gray")
    #axes[2].imshow(aug_image[..., 0], cmap="gray")
    #axes[3].imshow(aug_mask, cmap="gray")
    
    #plt.savefig(CFG.figures_dir + f'aug_fold_{CFG.valid_id}_{plot_count}.png')

    plot_count += 1
    if plot_count == 5:
        break
    
    

In [7]:
lit_model = UNET_TILE_lit(
        use_wandb = True,
        z_dim = Z_DIM,
        patch_size = (224,224),
        sw_batch_size=8 ,
        eta_min = 1e-7,
        t_max = 100,
        max_epochs = 1000,
        weight_decay =  0.001,
        learning_rate = 0.0001,
        gamma = 0.85,)


FROM_CHECKPOINT = False
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/Local_SMPEffB3_Tile_12_224patch/last.ckpt', 
                                              #patch_size = (512,512),
                                              learning_rate = 0.0001,
                                               weight_decay =  .001,
                                               eta_min = 1e-7,
                                             sw_batch_size = 8)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [ ]:
FROM_CHECKPOINT = False
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/Local_SMPEffB3_Tile_12_224patch/last.ckpt', 
                                              #patch_size = (512,512),
                                              learning_rate = 0.0001,
                                               weight_decay =  .001,
                                               eta_min = 1e-6,
                                             sw_batch_size = 8)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="fbeta_4",
    mode="max",
    dirpath="logs/AUG_BCE1_SMPEffB2_Tile_12_224patch",
    filename="AUG_BCE1_SMPEffB2_Tile_12_224patch{epoch:02d}{val_loss:.2f}{fbeta_4:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='mps',
        #benchmark=True,
        max_epochs=100,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/AUG_BCE1_SMPEffB2_Tile_12_224patch/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/AUG_BCE1_SMPEffB2_Tile_12_224patch/',
        #overfit_batches=1,
        #precision=16,
        accumulate_grad_batches=1, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Local_SMPEffB3_Tile_12_224patch/last.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-04 17:46:23,658 - GPU available: True (mps), used: True
2023-05-04 17:46:23,659 - TPU available: False, using: 0 TPU cores
2023-05-04 17:46:23,660 - IPU available: False, using: 0 IPUs
2023-05-04 17:46:23,660 - HPU available: False, using: 0 HPUs
Adjusting learning rate of group 0 to 1.0000e-04.
2023-05-04 17:46:23,880 - 
  | Name         | Type                  | Params
-------------------------------------------------------
0 | metrics      | ModuleDict            | 0     
1 | model        | Unet                  | 8.8 M 
2 | loss_dice    | DiceLoss              | 0     
3 | loss_tversky | TverskyLoss           | 0     
4 | loss_bce     | SoftBCEWithLogitsLoss | 0     
-------------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.034    Total estimated model params size (MB)


/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/gregory/PROJECT_ML/VESUVIUS_Challenge/logs/AUG_BCE1_SMPEffB2_Tile_12_224patch exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory logs/AUG_BCE1_SMPEffB2_Tile_12_224patch/lightning_logs/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(


Sanity Checking: 0it [00:00, ?it/s]

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/segmentation_models_pytorch/metrics/functional.py:214: UserWarning: MPS: no support for int64 for sum_out_mps, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1681456119295/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:132.)
  tp = (output * target).sum(2)
/Users/gregory/anaconda3/lib/python3.10/site-packages/torchmetrics/utilities/checks.py:57: UserWarning: MPS: no support for int64 for min_max, 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.6492e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.5892e-05.


Validation: 0it [00:00, ?it/s]